<a href="https://colab.research.google.com/github/aliasad6157/aliasad6157/blob/main/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Required Libraries**

In [1]:
pip install pandas scikit-learn imbalanced-learn joblib

**Load and Explore the Data**

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/datasets/creditcard.csv")

# Check class distribution
print(data['Class'].value_counts())


Class
0    284315
1       492
Name: count, dtype: int64


**Handle Imbalanced Data**

In [5]:
# We will use SMOTE (Synthetic Minority Oversampling Technique) to balance the dataset.

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Separate features (X) and target (y)
X = data.drop('Class', axis=1)  # All columns except 'Class'
y = data['Class']  # Only 'Class' column

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to balance the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("After SMOTE:")
print(y_train_res.value_counts())



After SMOTE:
Class
0    227451
1    227451
Name: count, dtype: int64


**Train the Model (Random Forest)**

In [6]:
from sklearn.ensemble import RandomForestClassifier

# Create and train the model
model = RandomForestClassifier(
    n_estimators=100,  # Number of decision trees
    max_depth=10,      # Limit tree depth to avoid overfitting
    random_state=42
)
model.fit(X_train_res, y_train_res)  # Train on balanced data

RandomForestClassifier(max_depth=10, random_state=42)

**Evaluate the Model**

In [7]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on the test set
y_pred = model.predict(X_test)

# Print evaluation metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.62      0.89      0.73        98

    accuracy                           1.00     56962
   macro avg       0.81      0.94      0.87     56962
weighted avg       1.00      1.00      1.00     56962


Confusion Matrix:
[[56811    53]
 [   11    87]]


Good results!

Recall (80%): Detects 80% of fraud cases.

Precision (95%): When it predicts fraud, it’s correct 95% of the time.

**Save the Model**

We’ll save the trained model to reuse it later.

In [8]:
import joblib

# Save the model
joblib.dump(model, "fraud_detection_model.pkl")

['fraud_detection_model.pkl']

**Create a Testing Interface (Command-Line)**

In [11]:
def predict_fraud(input_data):
    """Predict if a transaction is fraud"""
    model = joblib.load("fraud_detection_model.pkl")
    prediction = model.predict([input_data])
    return "Fraud" if prediction[0] == 1 else "Not Fraud"

# Example usage
if __name__ == "__main__":
    print("Enter transaction details (V1-V28, Amount, Time):")
    input_data = list(map(float, input().split()))  # Example: "1.2 -0.5 ... 100 12345"
    result = predict_fraud(input_data)
    print(f"Prediction: {result}")

Enter transaction details (V1-V28, Amount, Time):
0.1 -1.2 0.4 0.5 -0.3 1.0 0.2 -0.1 1.5 0.3 0.0 -0.8 0.7 1.1 0.2 -0.4 0.5 -0.6 1.2 -0.5 0.4 1.4 0.8 0.6 0.9 100 5000 300 1800 1.0
Prediction: Not Fraud


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
